In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
 
batch_size = 100

# transform
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5,), std=(0.5,))])
 
train_dataset = datasets.MNIST(root='./mnist_data/',
                              train=True,
                              transform=transform,
                              download=True)
 
test_dataset = datasets.MNIST(root='./mnist_data/',
                             train=False,
                             transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 27961778.63it/s]                            


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


32768it [00:00, 445877.14it/s]
  1%|          | 16384/1648877 [00:00<00:11, 144709.75it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw


1654784it [00:00, 7569734.82it/s]                            
8192it [00:00, 181953.52it/s]


Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw
Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw
Processing...
Done!


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm2d(num_features=64)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels = 64, kernel_size=5, stride = 1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels = 64, kernel_size=5, stride = 2)
        self.dropout1 = nn.Dropout(0.25)
        self.FC1 = nn.Linear(64 *12 * 12, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.FC2 = nn.Linear(128, 10)
        self.SM = nn.Softmax()
        
    def forward(self, x):
        in_size = x.size(0)
        
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(in_size, -1)
        x = self.dropout1(x)
        x = self.relu(self.FC1(x))
        x = self.dropout2(x)
        x = self.FC2(x)
        x = self.SM(x)

        return x

In [0]:
model = Net()
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        
        test_loss += criterion(output, target).item()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [4]:
for epoch in range(1, 11):
    train(epoch)	
    test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301852
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.301491
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.301596
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.301180
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.299456
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.299717
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.299759
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.300367
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.299311
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.298909
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.297832
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.299469
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.293286
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.293319
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.294775
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.289178
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.293475
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.291709
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.284063
Train Epoch: 1 [19000/60000 (32%)]	Loss: 2.290189
Train Epoch: 1 [200

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0167, Accuracy: 8099/10000 (81%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.744847
Train Epoch: 2 [1000/60000 (2%)]	Loss: 1.689337
Train Epoch: 2 [2000/60000 (3%)]	Loss: 1.661380
Train Epoch: 2 [3000/60000 (5%)]	Loss: 1.613014
Train Epoch: 2 [4000/60000 (7%)]	Loss: 1.757266
Train Epoch: 2 [5000/60000 (8%)]	Loss: 1.748191
Train Epoch: 2 [6000/60000 (10%)]	Loss: 1.715075
Train Epoch: 2 [7000/60000 (12%)]	Loss: 1.731076
Train Epoch: 2 [8000/60000 (13%)]	Loss: 1.751110
Train Epoch: 2 [9000/60000 (15%)]	Loss: 1.661626
Train Epoch: 2 [10000/60000 (17%)]	Loss: 1.689027
Train Epoch: 2 [11000/60000 (18%)]	Loss: 1.684732
Train Epoch: 2 [12000/60000 (20%)]	Loss: 1.712754
Train Epoch: 2 [13000/60000 (22%)]	Loss: 1.696103
Train Epoch: 2 [14000/60000 (23%)]	Loss: 1.619732
Train Epoch: 2 [15000/60000 (25%)]	Loss: 1.642039
Train Epoch: 2 [16000/60000 (27%)]	Loss: 1.704066
Train Epoch: 2 [17000/60000 (28%)]	Loss: 1.653655
Train Epoch: 2 [18000/60000 (30%)]	Loss: 1.648723
Train Ep

In [0]:
torch.save(model.state_dict(), "./checkpoint.pt")